<a href="https://colab.research.google.com/github/Toa-0128/2021-Census-of-Population-analysis/blob/main/consus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SECTION 1 — Install & Configure PySpark (Colab Setup)


In [ ]:
# Install Java (required for Apache Spark)
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq -y

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://cli.github.com/packages stable InRelease [3,917 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,623 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packag

Apache Spark 3.5.0 is already downloaded. Extracting the existing archive.

In [ ]:
# Extract Apache Spark 3.5.0 (Hadoop 3)
!tar -xzf spark-3.5.0-bin-hadoop3.tgz

In [ ]:
# Install findspark and pyspark
!pip install -q findspark pyspark

In [ ]:
# Set environment variables
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

import findspark
findspark.init()

from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Census Analysis Project") \
    .getOrCreate()


SECTION 2 — Load CSV Data into DataFrame

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Enter the path to your CSV file
csv_file_path = "/content/drive/MyDrive/98-401-X2021006_English_CSV_data_BritishColumbia.csv"

In [ ]:
if csv_file_path:
    try:
        # Read the CSV file into a PySpark DataFrame, inferring schema and including header
        df = spark.read.format("csv")\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .load(csv_file_path)

        print(f"Successfully loaded CSV from: {csv_file_path}")
        print("DataFrame Schema:")
        df.printSchema()
        print("First 5 rows of the DataFrame:")
        df.show(5)
    except Exception as e:
        print(f"Error loading CSV file: {e}")
else:
    print("Please provide a CSV file path in the previous cell.")

Successfully loaded CSV from: /content/drive/MyDrive/98-401-X2021006_English_CSV_data_BritishColumbia.csv
DataFrame Schema:
root
 |-- CENSUS_YEAR: integer (nullable = true)
 |-- DGUID: string (nullable = true)
 |-- ALT_GEO_CODE: integer (nullable = true)
 |-- GEO_LEVEL: string (nullable = true)
 |-- GEO_NAME: string (nullable = true)
 |-- TNR_SF: double (nullable = true)
 |-- TNR_LF: double (nullable = true)
 |-- DATA_QUALITY_FLAG: integer (nullable = true)
 |-- CHARACTERISTIC_ID: integer (nullable = true)
 |-- CHARACTERISTIC_NAME: string (nullable = true)
 |-- CHARACTERISTIC_NOTE: integer (nullable = true)
 |-- C1_COUNT_TOTAL: double (nullable = true)
 |-- SYMBOL12: string (nullable = true)
 |-- C2_COUNT_MEN+: double (nullable = true)
 |-- SYMBOL14: string (nullable = true)
 |-- C3_COUNT_WOMEN+: double (nullable = true)
 |-- SYMBOL16: string (nullable = true)
 |-- C10_RATE_TOTAL: double (nullable = true)
 |-- SYMBOL18: string (nullable = true)
 |-- C11_RATE_MEN+: double (nullable = tr

In [ ]:
#@title Enter the path to your NEW CSV file
csv_file_path2 = "/content/drive/MyDrive/98-401-X2021015_English_CSV_data.csv"

if csv_file_path2:
    try:
        # Read the NEW CSV file into a NEW PySpark DataFrame
        df_new = spark.read.format("csv")\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .load(csv_file_path2)

        print(f"Successfully loaded NEW CSV from: {csv_file_path2}")
        print("New DataFrame Schema:")
        df_new.printSchema()
        print("First 5 rows of the New DataFrame:")
        df_new.show(5)
    except Exception as e:
        print(f"Error loading new CSV file: {e}")
else:
    print("Please provide a new CSV file path in the previous cell.")

Successfully loaded NEW CSV from: /content/drive/MyDrive/98-401-X2021015_English_CSV_data.csv
New DataFrame Schema:
root
 |-- CENSUS_YEAR: integer (nullable = true)
 |-- DGUID: string (nullable = true)
 |-- ALT_GEO_CODE: integer (nullable = true)
 |-- GEO_LEVEL: string (nullable = true)
 |-- GEO_NAME: string (nullable = true)
 |-- TNR_SF: double (nullable = true)
 |-- TNR_LF: double (nullable = true)
 |-- DATA_QUALITY_FLAG: integer (nullable = true)
 |-- CHARACTERISTIC_ID: integer (nullable = true)
 |-- CHARACTERISTIC_NAME: string (nullable = true)
 |-- CHARACTERISTIC_NOTE: integer (nullable = true)
 |-- C1_COUNT_TOTAL: double (nullable = true)
 |-- SYMBOL12: string (nullable = true)
 |-- C2_COUNT_MEN+: double (nullable = true)
 |-- SYMBOL14: string (nullable = true)
 |-- C3_COUNT_WOMEN+: double (nullable = true)
 |-- SYMBOL16: string (nullable = true)
 |-- C10_RATE_TOTAL: double (nullable = true)
 |-- SYMBOL18: string (nullable = true)
 |-- C11_RATE_MEN+: double (nullable = true)
 |--